In [18]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn import *
import matplotlib.pyplot as plt
import numpy as np
import os

# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#!unzip -uq "/content/drive/My Drive/Colab Notebooks/dataset.zip" -d "/content/drive/My Drive/Colab Notebooks/dataset"
Directory=r"/content/drive/My Drive/Colab Notebooks/dataset/"
Category=["with_mask","without_mask"]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#grab the list of images

data=[]
category=[]


for cat in Category:
    path=os.path.join(Directory,cat)
    for p in os.listdir(path):
        img_path=os.path.join(path,p)
        image=load_img(img_path,target_size=(224,224))
        image=img_to_array(image)
        image=preprocess_input(image)
        
        data.append(image)
        category.append(cat)

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [5]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


#lb=LabelBinarizer()
lb = LabelBinarizer()
category=lb.fit_transform(category)
category=to_categorical(category)


## we have list of both data as well as for labels

## labels ----> categorical

In [6]:
data=np.array(data,dtype="float32")
category=np.array(category)

In [7]:
trainx,testx,trainy,testy=train_test_split(data,category,test_size=0.20,stratify=category,random_state=42)


In [8]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input,Flatten,Dropout,Conv2D,AveragePooling2D
from tensorflow.keras.optimizers import Adam

In [9]:
aug=ImageDataGenerator(rotation_range=20,
                       width_shift_range=0.20,
                       height_shift_range=0.20,
                       zoom_range=.15,
                       shear_range=0.15,
                       horizontal_flip=True,
                       fill_mode='nearest'
                       )

In [10]:
baseModel=MobileNetV2(weights='imagenet',
                      include_top=False,
                      input_tensor=Input(shape=(224,224,3)))

9412608/9406464 [==============================] - 0s 0us/step


In [11]:
from tensorflow.keras.layers import Dense
headModel=baseModel.output
headModel=AveragePooling2D(pool_size=(7,7))(headModel)
headModel=Flatten(name='flatten')(headModel)
headModel=Dense(128,activation='relu')(headModel)
headModel=Dropout(0.5)(headModel)
headModel=Dense(2,activation='softmax')(headModel)

In [12]:
## now place this FC on the top of baseModel

model=Model(inputs=baseModel.input,outputs=headModel)

In [13]:
for layer in baseModel.layers:
  layer.trainable=False

In [14]:
model.compile(optimizer="Adam",loss="binary_crossentropy",metrics=['accuracy'])

In [15]:
model.fit(aug.flow(trainx,trainy,batch_size=32),steps_per_epoch=len(trainx)//32,validation_data=(testx,testy),validation_steps=len(testx)//32,epochs=20)

Epoch 1/20
54/54 [==============================] - 84s 2s/step - loss: 0.0980 - accuracy: 0.9678 - val_loss: 0.0377 - val_accuracy: 0.9885
Epoch 2/20
54/54 [==============================] - 83s 2s/step - loss: 0.0332 - accuracy: 0.9901 - val_loss: 0.0556 - val_accuracy: 0.9862
Epoch 3/20
54/54 [==============================] - 83s 2s/step - loss: 0.0208 - accuracy: 0.9901 - val_loss: 0.0608 - val_accuracy: 0.9885
Epoch 4/20
54/54 [==============================] - 83s 2s/step - loss: 0.0170 - accuracy: 0.9947 - val_loss: 0.0464 - val_accuracy: 0.9862
Epoch 5/20
54/54 [==============================] - 83s 2s/step - loss: 0.0213 - accuracy: 0.9918 - val_loss: 0.0416 - val_accuracy: 0.9885
Epoch 6/20
54/54 [==============================] - 86s 2s/step - loss: 0.0205 - accuracy: 0.9936 - val_loss: 0.0290 - val_accuracy: 0.9931
Epoch 7/20
54/54 [==============================] - 84s 2s/step - loss: 0.0141 - accuracy: 0.9965 - val_loss: 0.0374 - val_accuracy: 0.9862
Epoch 8/20
54/54 [==

In [16]:
model.save("mask_detector.model",save_format="h5")